## augmentation을 이용한 CNN 모델 학습

### → 과대적합 방지책

- 데이터셋 : [Dogs vs. Cats dataset](https://www.kaggle.com/c/dogs-vs-cats/data)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
tf.__version__

### 데이터 증식 사용

- 과대적합 : 학습할 샘플이 너무 적은 경우 새로운 데이터에 일반화할 수 있는 모델을 훈련시킬 수 없어 발생
- 데이터 증식 : 기존 훈련 샘플로부터 더 많은 훈련 데이터를 생성


- **ImageDataGenerator()**를 이용하여 데이터 증식 : 여러 종류의 랜덤 변환을 적용하도록 설정가능
  - rotation_range80)
  - width_shift_range
  - height_shift_range
  - shear_range
  - zoom_range
  - horizontal_flip 
  - fill_mode 

### 데이터 준비

- 디렉토리 설정: 이미지 데이터가 있는 경로

In [ ]:
base_dir = './datasets/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

### 이미지 데이터 증식 샘플 
1. 데이터 증식 설정
2. 랜덤하게 증식된 훈련 이미지 그리기

#### 1. 데이터 증식 설정 : ImageDataGenerator() 사용

In [ ]:
# ImageDataGenerator를 사용하여 데이터 증식 설정
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

#### 2. 랜덤하게 증식된 훈련 이미지 그리기

In [ ]:
# 랜덤하게 증식된 훈련 이미지 그리기
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

fnames = sorted([os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)])

# 증식할 이미지 선택
img_path = fnames[77] 
img = image.load_img(img_path, target_size=(150, 150))

x = image.img_to_array(img)   # (150,150,3) 크기의 넘파이 배열로 변환
x = x.reshape((1,)+x.shape)   # (1, 150, 150, 3) 크기로 변환

# 랜덤하게 변환된 이미지 배치를 생성
# 무한 반복되므로 중지 시점 정해야 함:4개 이미지 생성하고 중단 
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i%4 == 0:
        break       

plt.show()


## 드롭아웃 층 포함한 새로운 ConvNet 정의 및 훈련
- 과대적합을 억제하기 위해 완전 연결 분류기 직전에 Dropout층을 추가함
- 데이터 증식 제너레이터를 사용하여 컨브넷 훈련 

### 1. 이미지 증식(Image Augmentation)

- Normalization & Augmentation
    - ImageDataGenerator( )


- Resizing & Generator
    - flow_from_directory( )

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

# 검증데이터, 테스트 데이터는 증식되어서는 안됨
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'
)

### 2. 모델 정의

In [ ]:
# 드롭아웃을 포함한 새로운 컨브넷 정의하기
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

### 3. 모델 컴파일

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

### 4. 모델 학습
- GPU 기반: 48분 소요
    - epochs : 60 -> 100

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50
)

### 5. 모델 학습 결과 시각화
- 데이터 증식과 드롭아웃을 적용한 모델의 훈련과 검증데이터의 정확도와 손실 그래프

In [ ]:
# 데이터 증식과 드롭아웃을 적용한 모델의 훈련과 검증데이터의 정확도 그래프

acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(9,6))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# 데이터 증식과 드롭아웃을 적용한 모델의 훈련과 검증데이터의 손실 그래프

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

### 6. 모델 평가
 - test_generator

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
                 test_dir,
                 target_size = (150, 150),
                 batch_size = 20,
                 class_mode = 'binary')

In [ ]:
loss, accuracy = model.evaluate(test_generator,
                                       steps = 50)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

### 7. 모델 저장

In [ ]:
model.save('./model/cats_and_dogs_small_augmentation.h5')

In [ ]:
!ls -l /model

### 8. 모델 로드(load)

- 저장된 모델 로드

In [ ]:
from tensorflow.keras.models import load_model

path = './model/cats_and_dogs_small_augmentation.h5'
model_google = load_model(path)

- Loss & Accuracy

In [ ]:
loss, accuracy = model_google.evaluate(test_generator,
                                       steps = 50)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))